In [50]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import os
import pandas as pd
from shutil import copyfile
import random
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
working_dir = os.getcwd()
print(working_dir)
# dataframe for train.csv
train_df = pd.read_csv(working_dir+'/train.csv')
# dataframe for test.csv
test_df = pd.read_csv(working_dir+'/test.csv')

G:\hackathon_1


In [3]:
print(train_df.head())
print(test_df.head())

  image_names  emergency_or_not
0    1503.jpg                 0
1    1420.jpg                 0
2    1764.jpg                 0
3    1356.jpg                 0
4    1117.jpg                 0
  image_names
0    1960.jpg
1     668.jpg
2    2082.jpg
3     808.jpg
4    1907.jpg


In [4]:
# 2352 total images
print(len(os.listdir(working_dir+'/images')))

2352


In [5]:
# create folders
to_create = [
    working_dir+'/emer_source',
    working_dir+'/nonemer_source',
    working_dir+'/tmp',
    working_dir+'/tmp/emer-v-nonemer',
    working_dir+'/tmp/emer-v-nonemer/training',
    working_dir+'/tmp/emer-v-nonemer/testing',
    working_dir+'/tmp/emer-v-nonemer/training/emer',
    working_dir+'/tmp/emer-v-nonemer/training/nonemer',
    working_dir+'/tmp/emer-v-nonemer/testing/emer',
    working_dir+'/tmp/emer-v-nonemer/testing/nonemer'
]

for directory in to_create:
    try:
        os.mkdir(directory)
        print(directory, 'created')
    except:
        print(directory, 'failed')

G:\hackathon_1/tmp failed
G:\hackathon_1/tmp/emer-v-nonemer failed
G:\hackathon_1/tmp/emer-v-nonemer/training failed
G:\hackathon_1/tmp/emer-v-nonemer/testing failed
G:\hackathon_1/tmp/emer-v-nonemer/training/emer failed
G:\hackathon_1/tmp/emer-v-nonemer/training/nonemer failed
G:\hackathon_1/tmp/emer-v-nonemer/testing/emer failed
G:\hackathon_1/tmp/emer-v-nonemer/testing/nonemer failed


In [36]:
emer_training = working_dir+'/tmp/emer-v-nonemer/training/emer/'
nonemer_training = working_dir+'/tmp/emer-v-nonemer/training/nonemer/'
emer_testing = working_dir+'/tmp/emer-v-nonemer/testing/emer/'
nonemer_testing = working_dir+'/tmp/emer-v-nonemer/testing/nonemer/'

In [16]:
print(train_df['image_names'][0])
print(train_df['emergency_or_not'][0])
print(test_df['image_names'][0])

print(train_df.iloc[0])

1503.jpg
0
1960.jpg
image_names         1503.jpg
emergency_or_not           0
Name: 0, dtype: object


In [22]:
# separate dataframes for emergency and nonemergency images in training set
training_emer = train_df.loc[train_df['emergency_or_not']==0]
training_nonemer = train_df.loc[train_df['emergency_or_not']==1]

In [28]:
# # Move the labelled files into a separate folder for training and validation files
# for i in train_df['image_names']:
#     image_name = i
#     copyfile(working_dir+'/images/'+i, working_dir+'/trainValImages/'+i)

In [31]:
for i in training_emer['image_names']:
    image_name = i
    copyfile(working_dir+'/images/'+i, working_dir+'/emer_source/'+i)
    
for i in training_nonemer['image_names']:
    image_name = i
    copyfile(working_dir+'/images/'+i, working_dir+'/nonemer_source/'+i)

In [37]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    all_files = []
    
    for file_name in os.listdir(SOURCE):
        file_path = SOURCE + file_name

        if os.path.getsize(file_path):
            all_files.append(file_name)
        else:
            print('{} is zero length, so ignoring'.format(file_name))
    
    n_files = len(all_files)
    split_point = int(n_files * SPLIT_SIZE)
    
    shuffled = random.sample(all_files, n_files)
    
    train_set = shuffled[:split_point]
    test_set = shuffled[split_point:]
    
    for file_name in train_set:
        copyfile(SOURCE + file_name, TRAINING + file_name)
        
    for file_name in test_set:
        copyfile(SOURCE + file_name, TESTING + file_name)

emer_source_dir = (working_dir+'/emer_source/')
train_emer_dir = emer_training
test_emer_dir = emer_testing

nonemer_source_dir = (working_dir+'/nonemer_source/')
train_nonemer_dir = nonemer_training
test_nonemer_dir = nonemer_testing

split_size = .9
split_data(emer_source_dir, train_emer_dir, test_emer_dir, split_size)
split_data(nonemer_source_dir, train_nonemer_dir, test_nonemer_dir, split_size)

In [38]:
print(len(os.listdir(emer_source_dir)))
print(len(os.listdir(nonemer_source_dir)))

965
681


In [40]:
print(len(os.listdir(train_emer_dir)))
print(len(os.listdir(test_emer_dir)))
print('-----')
print(len(os.listdir(train_nonemer_dir)))
print(len(os.listdir(test_nonemer_dir)))

868
97
-----
612
69


In [41]:
import cv2
image = cv2.imread(working_dir+'/images/0.jpg')

In [44]:
cv2.imshow('title',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
print(image.shape)

(224, 224, 3)


In [48]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (2,2), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [51]:
TRAINING_DIR = working_dir+'/tmp/emer-v-nonemer/training'
train_datagen = ImageDataGenerator(rescale = 1 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=64,
    class_mode='binary',
    target_size=(224, 224)
)

VALIDATION_DIR = working_dir+'/tmp/emer-v-nonemer/testing'
validation_datagen = ImageDataGenerator(rescale= 1/255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=64,
    class_mode='binary',
    target_size=(224, 224)
)

Found 1480 images belonging to 2 classes.
Found 166 images belonging to 2 classes.


In [52]:
import warnings
warnings.filterwarnings('ignore')
print('warnings ignored')

warnings ignored


In [53]:
history = model.fit_generator(
    train_generator,
    epochs=5,
    verbose=1,
    validation_data=validation_generator
)

Epoch 1/5


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-5f243048186a>", line 5, in <module>
    validation_data=validation_generator
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 1297, in fit_generator
    steps_name='steps_per_epoch')
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_generator.py", line 265, in model_iteration
    batch_outs = batch_function(*batch_data)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 973, in train_on_batch
    class_weight=class_weight, reset_metrics=reset_metrics)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2_utils.py", line 264, in train_on_batch
    output_loss_metrics=

KeyboardInterrupt: 